<a href="https://colab.research.google.com/github/NadiaSouza1304/Hack_DS_01/blob/credit_card_balance/Hackathon_DS/HackathonCD_Feature_Engineering_Credit_Card_Balance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [72]:
!pip install pyspark

In [73]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
!tar xf spark-3.1.2-bin-hadoop3.2.tgz
!pip install -q findspark

In [74]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"

In [75]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
from pyspark.sql.functions import lit, current_timestamp, date_format, col, to_timestamp

In [76]:
spark.conf.set("spark.sql.session.timeZone", "UTC-3")

# **Lendo as bases com Spark**

In [77]:
credit_card_balance = spark.read.csv("/content/drive/MyDrive/hackathon-pod-ds/credit_card_balance.csv",sep=',',header=True, encoding='utf-8', inferSchema=True)
credit_card_balance.createOrReplaceTempView("credit_card_balance")

# **Vamos ver quantos IDs da CREDIT CARD BALANCE são únicos**

In [79]:
query = """
    SELECT
        COUNT(DISTINCT(SK_ID_CURR)) AS sk_id_curr_unique_credit
    FROM
        credit_card_balance
"""

# Execute a consulta
id_occurrences = spark.sql(query)

# Mostre os resultados
id_occurrences.show()

+------------------------+
|sk_id_curr_unique_credit|
+------------------------+
|                  103558|
+------------------------+



# **Vamos ver quantos linhas temos na CREDIT CARD BALANCE (considerando repetidas)**

In [80]:
credit_card_balance.count()

3840312

# **Vamos ver os tipos de cada variável na CREDIT CARD BALANCE**

In [81]:
credit_card_balance.printSchema()

root
 |-- SK_ID_PREV: integer (nullable = true)
 |-- SK_ID_CURR: integer (nullable = true)
 |-- MONTHS_BALANCE: integer (nullable = true)
 |-- AMT_BALANCE: double (nullable = true)
 |-- AMT_CREDIT_LIMIT_ACTUAL: integer (nullable = true)
 |-- AMT_DRAWINGS_ATM_CURRENT: double (nullable = true)
 |-- AMT_DRAWINGS_CURRENT: double (nullable = true)
 |-- AMT_DRAWINGS_OTHER_CURRENT: double (nullable = true)
 |-- AMT_DRAWINGS_POS_CURRENT: double (nullable = true)
 |-- AMT_INST_MIN_REGULARITY: double (nullable = true)
 |-- AMT_PAYMENT_CURRENT: double (nullable = true)
 |-- AMT_PAYMENT_TOTAL_CURRENT: double (nullable = true)
 |-- AMT_RECEIVABLE_PRINCIPAL: double (nullable = true)
 |-- AMT_RECIVABLE: double (nullable = true)
 |-- AMT_TOTAL_RECEIVABLE: double (nullable = true)
 |-- CNT_DRAWINGS_ATM_CURRENT: double (nullable = true)
 |-- CNT_DRAWINGS_CURRENT: integer (nullable = true)
 |-- CNT_DRAWINGS_OTHER_CURRENT: double (nullable = true)
 |-- CNT_DRAWINGS_POS_CURRENT: double (nullable = true)
 |

# **Vamos ver quantos IDs da APPLICATION TRAIN são únicos**

In [82]:
query2 = """
    SELECT
        COUNT(DISTINCT(SK_ID_CURR)) AS sk_id_curr_unique_application_train
    FROM
        application_train
"""

# Execute a consulta
id_occurrences2 = spark.sql(query2)

# Mostre os resultados
id_occurrences2.show()

+-----------------------------------+
|sk_id_curr_unique_application_train|
+-----------------------------------+
|                             215257|
+-----------------------------------+



# **Vamos ver quantos linhas temos na APPLICATION TRAIN (considerando repetidas)**

In [83]:
application_train.count()

215257

# **Vamos ver quais categorias temos na única variável categórica da CREDIT CARD BALANCE (considerando repetidas)**

In [84]:
spark.sql("""
SELECT
  NAME_CONTRACT_STATUS,
  Count(*)
  FROM
  credit_card_balance
  GROUP BY NAME_CONTRACT_STATUS
  ORDER BY Count(*)
""").show()

+--------------------+--------+
|NAME_CONTRACT_STATUS|count(1)|
+--------------------+--------+
|            Approved|       5|
|             Refused|      17|
|       Sent proposal|     513|
|              Demand|    1365|
|              Signed|   11058|
|           Completed|  128918|
|              Active| 3698436|
+--------------------+--------+



# **1º DF: Transformando a variável categórica em variável numérica**

A coluna categórica dessa tabela é a NAME_CONTRACT_STATUS.

In [93]:
df_credit_card_balance_cat = spark.sql("""
SELECT
  SK_ID_CURR,
  ROUND(SUM(CASE WHEN NAME_CONTRACT_STATUS = 'Approved' THEN 1 ELSE 0 END), 2) AS qtd_name_contract_status_approved,
  ROUND(SUM(CASE WHEN NAME_CONTRACT_STATUS = 'Refused' THEN 1 ELSE 0 END), 2) AS qtd_name_contract_status_refused,
  ROUND(SUM(CASE WHEN NAME_CONTRACT_STATUS = 'Sent proposal' THEN 1 ELSE 0 END), 2) AS qtd_name_contract_status_sent_proposal,
  ROUND(SUM(CASE WHEN NAME_CONTRACT_STATUS = 'Demand' THEN 1 ELSE 0 END), 2) AS qtd_name_contract_status_demand,
  ROUND(SUM(CASE WHEN NAME_CONTRACT_STATUS = 'Signed' THEN 1 ELSE 0 END), 2) AS qtd_name_contract_status_signed,
  ROUND(SUM(CASE WHEN NAME_CONTRACT_STATUS = 'Completed proposal' THEN 1 ELSE 0 END), 2) AS qtd_name_contract_status_completed,
  ROUND(SUM(CASE WHEN NAME_CONTRACT_STATUS = 'Active' THEN 1 ELSE 0 END), 2) AS qtd_name_contract_status_active,
  ROUND(MIN(CASE WHEN NAME_CONTRACT_STATUS = 'Approved' THEN 1 ELSE 0 END), 2) AS min_name_contract_status_approved,
  ROUND(MIN(CASE WHEN NAME_CONTRACT_STATUS = 'Refused' THEN 1 ELSE 0 END), 2) AS min_name_contract_status_refused,
  ROUND(MIN(CASE WHEN NAME_CONTRACT_STATUS = 'Sent proposal' THEN 1 ELSE 0 END), 2) AS min_name_contract_status_sent_proposal,
  ROUND(MIN(CASE WHEN NAME_CONTRACT_STATUS = 'Demand' THEN 1 ELSE 0 END), 2) AS min_name_contract_status_demand,
  ROUND(MIN(CASE WHEN NAME_CONTRACT_STATUS = 'Signed' THEN 1 ELSE 0 END), 2) AS min_name_contract_status_signed,
  ROUND(MIN(CASE WHEN NAME_CONTRACT_STATUS = 'Completed proposal' THEN 1 ELSE 0 END), 2) AS min_name_contract_status_completed,
  ROUND(MIN(CASE WHEN NAME_CONTRACT_STATUS = 'Active' THEN 1 ELSE 0 END), 2) AS min_name_contract_status_active,
  ROUND(MAX(CASE WHEN NAME_CONTRACT_STATUS = 'Approved' THEN 1 ELSE 0 END), 2) AS max_name_contract_status_approved,
  ROUND(MAX(CASE WHEN NAME_CONTRACT_STATUS = 'Refused' THEN 1 ELSE 0 END), 2) AS max_name_contract_status_refused,
  ROUND(MAX(CASE WHEN NAME_CONTRACT_STATUS = 'Sent proposal' THEN 1 ELSE 0 END), 2) AS max_name_contract_status_sent_proposal,
  ROUND(MAX(CASE WHEN NAME_CONTRACT_STATUS = 'Demand' THEN 1 ELSE 0 END), 2) AS max_name_contract_status_demand,
  ROUND(MAX(CASE WHEN NAME_CONTRACT_STATUS = 'Signed' THEN 1 ELSE 0 END), 2) AS max_name_contract_status_signed,
  ROUND(MAX(CASE WHEN NAME_CONTRACT_STATUS = 'Completed proposal' THEN 1 ELSE 0 END), 2) AS max_name_contract_status_completed,
  ROUND(MAX(CASE WHEN NAME_CONTRACT_STATUS = 'Active' THEN 1 ELSE 0 END), 2) AS max_name_contract_status_active,
  ROUND(AVG(CASE WHEN NAME_CONTRACT_STATUS = 'Approved' THEN 1 ELSE 0 END), 2) AS avg_name_contract_status_approved,
  ROUND(AVG(CASE WHEN NAME_CONTRACT_STATUS = 'Refused' THEN 1 ELSE 0 END), 2) AS avg_name_contract_status_refused,
  ROUND(AVG(CASE WHEN NAME_CONTRACT_STATUS = 'Sent proposal' THEN 1 ELSE 0 END), 2) AS avg_name_contract_status_sent_proposal,
  ROUND(AVG(CASE WHEN NAME_CONTRACT_STATUS = 'Demand' THEN 1 ELSE 0 END), 2) AS avg_name_contract_status_demand,
  ROUND(AVG(CASE WHEN NAME_CONTRACT_STATUS = 'Signed' THEN 1 ELSE 0 END), 2) AS avg_name_contract_status_signed,
  ROUND(AVG(CASE WHEN NAME_CONTRACT_STATUS = 'Completed proposal' THEN 1 ELSE 0 END), 2) AS avg_name_contract_status_completed,
  ROUND(AVG(CASE WHEN NAME_CONTRACT_STATUS = 'Active' THEN 1 ELSE 0 END), 2) AS avg_name_contract_status_active
FROM
  credit_card_balance
GROUP BY
  SK_ID_CURR
""")



In [95]:
df_credit_card_balance_cat.show()

+----------+---------------------------------+--------------------------------+--------------------------------------+-------------------------------+-------------------------------+----------------------------------+-------------------------------+---------------------------------+--------------------------------+--------------------------------------+-------------------------------+-------------------------------+----------------------------------+-------------------------------+---------------------------------+--------------------------------+--------------------------------------+-------------------------------+-------------------------------+----------------------------------+-------------------------------+---------------------------------+--------------------------------+--------------------------------------+-------------------------------+-------------------------------+----------------------------------+-------------------------------+
|SK_ID_CURR|qtd_name_contract_status_ap

In [94]:
df_credit_card_balance_cat.createOrReplaceTempView("df_credit_card_balance_cat")

# **2º DF: Transformando as variáveis numéricas em agregações**

As colunas numéricas e de datas são:

1. MONTHS_BALANCE: integer
2. AMT_BALANCE: double
3. AMT_CREDIT_LIMIT_ACTUAL: integer
4. AMT_DRAWINGS_ATM_CURRENT: double
5. AMT_DRAWINGS_CURRENT: double
6. AMT_DRAWINGS_OTHER_CURRENT: double
7. AMT_DRAWINGS_POS_CURRENT: double
8. AMT_INST_MIN_REGULARITY: double
9. AMT_PAYMENT_CURRENT: double
10. AMT_PAYMENT_TOTAL_CURRENT: double
11. AMT_RECEIVABLE_PRINCIPAL: double
12. AMT_RECIVABLE: double
13. AMT_TOTAL_RECEIVABLE: double
14. CNT_DRAWINGS_ATM_CURRENT: double
15. CNT_DRAWINGS_CURRENT: integer
16. CNT_DRAWINGS_OTHER_CURRENT: double
17. CNT_DRAWINGS_POS_CURRENT: double
18. CNT_INSTALMENT_MATURE_CUM: double


In [96]:
# Criando as agregações

agg = ['SUM', 'MIN', 'MAX', 'AVG']

# Criando a variável numérica e de data

var_num = ['MONTHS_BALANCE', 'AMT_BALANCE', 'AMT_CREDIT_LIMIT_ACTUAL', \
           'AMT_DRAWINGS_ATM_CURRENT', 'AMT_DRAWINGS_CURRENT', \
           'AMT_DRAWINGS_OTHER_CURRENT', 'AMT_DRAWINGS_POS_CURRENT', 'AMT_INST_MIN_REGULARITY',
           'AMT_PAYMENT_CURRENT', 'AMT_PAYMENT_TOTAL_CURRENT', 'AMT_RECEIVABLE_PRINCIPAL', \
           'AMT_RECIVABLE', 'AMT_TOTAL_RECEIVABLE', 'CNT_DRAWINGS_ATM_CURRENT', 'CNT_DRAWINGS_CURRENT', \
           'CNT_DRAWINGS_OTHER_CURRENT', 'CNT_DRAWINGS_POS_CURRENT', 'CNT_INSTALMENT_MATURE_CUM']

In [97]:
# Criando uma rotina que gera o texto da função de agregação

for ag in agg:
  for var_n in var_num:
        print(ag + "(" + var_n + ") AS " + ag + "_" + var_n + ',')

SUM(MONTHS_BALANCE) AS SUM_MONTHS_BALANCE,
SUM(AMT_BALANCE) AS SUM_AMT_BALANCE,
SUM(AMT_CREDIT_LIMIT_ACTUAL) AS SUM_AMT_CREDIT_LIMIT_ACTUAL,
SUM(AMT_DRAWINGS_ATM_CURRENT) AS SUM_AMT_DRAWINGS_ATM_CURRENT,
SUM(AMT_DRAWINGS_CURRENT) AS SUM_AMT_DRAWINGS_CURRENT,
SUM(AMT_DRAWINGS_OTHER_CURRENT) AS SUM_AMT_DRAWINGS_OTHER_CURRENT,
SUM(AMT_DRAWINGS_POS_CURRENT) AS SUM_AMT_DRAWINGS_POS_CURRENT,
SUM(AMT_INST_MIN_REGULARITY) AS SUM_AMT_INST_MIN_REGULARITY,
SUM(AMT_PAYMENT_CURRENT) AS SUM_AMT_PAYMENT_CURRENT,
SUM(AMT_PAYMENT_TOTAL_CURRENT) AS SUM_AMT_PAYMENT_TOTAL_CURRENT,
SUM(AMT_RECEIVABLE_PRINCIPAL) AS SUM_AMT_RECEIVABLE_PRINCIPAL,
SUM(AMT_RECIVABLE) AS SUM_AMT_RECIVABLE,
SUM(AMT_TOTAL_RECEIVABLE) AS SUM_AMT_TOTAL_RECEIVABLE,
SUM(CNT_DRAWINGS_ATM_CURRENT) AS SUM_CNT_DRAWINGS_ATM_CURRENT,
SUM(CNT_DRAWINGS_CURRENT) AS SUM_CNT_DRAWINGS_CURRENT,
SUM(CNT_DRAWINGS_OTHER_CURRENT) AS SUM_CNT_DRAWINGS_OTHER_CURRENT,
SUM(CNT_DRAWINGS_POS_CURRENT) AS SUM_CNT_DRAWINGS_POS_CURRENT,
SUM(CNT_INSTALMENT_MATURE

In [98]:
df_credit_card_balance_num = spark.sql("""
SELECT
  SK_ID_CURR,
SUM(MONTHS_BALANCE) AS SUM_MONTHS_BALANCE,
SUM(AMT_BALANCE) AS SUM_AMT_BALANCE,
SUM(AMT_CREDIT_LIMIT_ACTUAL) AS SUM_AMT_CREDIT_LIMIT_ACTUAL,
SUM(AMT_DRAWINGS_ATM_CURRENT) AS SUM_AMT_DRAWINGS_ATM_CURRENT,
SUM(AMT_DRAWINGS_CURRENT) AS SUM_AMT_DRAWINGS_CURRENT,
SUM(AMT_DRAWINGS_OTHER_CURRENT) AS SUM_AMT_DRAWINGS_OTHER_CURRENT,
SUM(AMT_DRAWINGS_POS_CURRENT) AS SUM_AMT_DRAWINGS_POS_CURRENT,
SUM(AMT_INST_MIN_REGULARITY) AS SUM_AMT_INST_MIN_REGULARITY,
SUM(AMT_PAYMENT_CURRENT) AS SUM_AMT_PAYMENT_CURRENT,
SUM(AMT_PAYMENT_TOTAL_CURRENT) AS SUM_AMT_PAYMENT_TOTAL_CURRENT,
SUM(AMT_RECEIVABLE_PRINCIPAL) AS SUM_AMT_RECEIVABLE_PRINCIPAL,
SUM(AMT_RECIVABLE) AS SUM_AMT_RECIVABLE,
SUM(AMT_TOTAL_RECEIVABLE) AS SUM_AMT_TOTAL_RECEIVABLE,
SUM(CNT_DRAWINGS_ATM_CURRENT) AS SUM_CNT_DRAWINGS_ATM_CURRENT,
SUM(CNT_DRAWINGS_CURRENT) AS SUM_CNT_DRAWINGS_CURRENT,
SUM(CNT_DRAWINGS_OTHER_CURRENT) AS SUM_CNT_DRAWINGS_OTHER_CURRENT,
SUM(CNT_DRAWINGS_POS_CURRENT) AS SUM_CNT_DRAWINGS_POS_CURRENT,
SUM(CNT_INSTALMENT_MATURE_CUM) AS SUM_CNT_INSTALMENT_MATURE_CUM,
MIN(MONTHS_BALANCE) AS MIN_MONTHS_BALANCE,
MIN(AMT_BALANCE) AS MIN_AMT_BALANCE,
MIN(AMT_CREDIT_LIMIT_ACTUAL) AS MIN_AMT_CREDIT_LIMIT_ACTUAL,
MIN(AMT_DRAWINGS_ATM_CURRENT) AS MIN_AMT_DRAWINGS_ATM_CURRENT,
MIN(AMT_DRAWINGS_CURRENT) AS MIN_AMT_DRAWINGS_CURRENT,
MIN(AMT_DRAWINGS_OTHER_CURRENT) AS MIN_AMT_DRAWINGS_OTHER_CURRENT,
MIN(AMT_DRAWINGS_POS_CURRENT) AS MIN_AMT_DRAWINGS_POS_CURRENT,
MIN(AMT_INST_MIN_REGULARITY) AS MIN_AMT_INST_MIN_REGULARITY,
MIN(AMT_PAYMENT_CURRENT) AS MIN_AMT_PAYMENT_CURRENT,
MIN(AMT_PAYMENT_TOTAL_CURRENT) AS MIN_AMT_PAYMENT_TOTAL_CURRENT,
MIN(AMT_RECEIVABLE_PRINCIPAL) AS MIN_AMT_RECEIVABLE_PRINCIPAL,
MIN(AMT_RECIVABLE) AS MIN_AMT_RECIVABLE,
MIN(AMT_TOTAL_RECEIVABLE) AS MIN_AMT_TOTAL_RECEIVABLE,
MIN(CNT_DRAWINGS_ATM_CURRENT) AS MIN_CNT_DRAWINGS_ATM_CURRENT,
MIN(CNT_DRAWINGS_CURRENT) AS MIN_CNT_DRAWINGS_CURRENT,
MIN(CNT_DRAWINGS_OTHER_CURRENT) AS MIN_CNT_DRAWINGS_OTHER_CURRENT,
MIN(CNT_DRAWINGS_POS_CURRENT) AS MIN_CNT_DRAWINGS_POS_CURRENT,
MIN(CNT_INSTALMENT_MATURE_CUM) AS MIN_CNT_INSTALMENT_MATURE_CUM,
MAX(MONTHS_BALANCE) AS MAX_MONTHS_BALANCE,
MAX(AMT_BALANCE) AS MAX_AMT_BALANCE,
MAX(AMT_CREDIT_LIMIT_ACTUAL) AS MAX_AMT_CREDIT_LIMIT_ACTUAL,
MAX(AMT_DRAWINGS_ATM_CURRENT) AS MAX_AMT_DRAWINGS_ATM_CURRENT,
MAX(AMT_DRAWINGS_CURRENT) AS MAX_AMT_DRAWINGS_CURRENT,
MAX(AMT_DRAWINGS_OTHER_CURRENT) AS MAX_AMT_DRAWINGS_OTHER_CURRENT,
MAX(AMT_DRAWINGS_POS_CURRENT) AS MAX_AMT_DRAWINGS_POS_CURRENT,
MAX(AMT_INST_MIN_REGULARITY) AS MAX_AMT_INST_MIN_REGULARITY,
MAX(AMT_PAYMENT_CURRENT) AS MAX_AMT_PAYMENT_CURRENT,
MAX(AMT_PAYMENT_TOTAL_CURRENT) AS MAX_AMT_PAYMENT_TOTAL_CURRENT,
MAX(AMT_RECEIVABLE_PRINCIPAL) AS MAX_AMT_RECEIVABLE_PRINCIPAL,
MAX(AMT_RECIVABLE) AS MAX_AMT_RECIVABLE,
MAX(AMT_TOTAL_RECEIVABLE) AS MAX_AMT_TOTAL_RECEIVABLE,
MAX(CNT_DRAWINGS_ATM_CURRENT) AS MAX_CNT_DRAWINGS_ATM_CURRENT,
MAX(CNT_DRAWINGS_CURRENT) AS MAX_CNT_DRAWINGS_CURRENT,
MAX(CNT_DRAWINGS_OTHER_CURRENT) AS MAX_CNT_DRAWINGS_OTHER_CURRENT,
MAX(CNT_DRAWINGS_POS_CURRENT) AS MAX_CNT_DRAWINGS_POS_CURRENT,
MAX(CNT_INSTALMENT_MATURE_CUM) AS MAX_CNT_INSTALMENT_MATURE_CUM,
AVG(MONTHS_BALANCE) AS AVG_MONTHS_BALANCE,
AVG(AMT_BALANCE) AS AVG_AMT_BALANCE,
AVG(AMT_CREDIT_LIMIT_ACTUAL) AS AVG_AMT_CREDIT_LIMIT_ACTUAL,
AVG(AMT_DRAWINGS_ATM_CURRENT) AS AVG_AMT_DRAWINGS_ATM_CURRENT,
AVG(AMT_DRAWINGS_CURRENT) AS AVG_AMT_DRAWINGS_CURRENT,
AVG(AMT_DRAWINGS_OTHER_CURRENT) AS AVG_AMT_DRAWINGS_OTHER_CURRENT,
AVG(AMT_DRAWINGS_POS_CURRENT) AS AVG_AMT_DRAWINGS_POS_CURRENT,
AVG(AMT_INST_MIN_REGULARITY) AS AVG_AMT_INST_MIN_REGULARITY,
AVG(AMT_PAYMENT_CURRENT) AS AVG_AMT_PAYMENT_CURRENT,
AVG(AMT_PAYMENT_TOTAL_CURRENT) AS AVG_AMT_PAYMENT_TOTAL_CURRENT,
AVG(AMT_RECEIVABLE_PRINCIPAL) AS AVG_AMT_RECEIVABLE_PRINCIPAL,
AVG(AMT_RECIVABLE) AS AVG_AMT_RECIVABLE,
AVG(AMT_TOTAL_RECEIVABLE) AS AVG_AMT_TOTAL_RECEIVABLE,
AVG(CNT_DRAWINGS_ATM_CURRENT) AS AVG_CNT_DRAWINGS_ATM_CURRENT,
AVG(CNT_DRAWINGS_CURRENT) AS AVG_CNT_DRAWINGS_CURRENT,
AVG(CNT_DRAWINGS_OTHER_CURRENT) AS AVG_CNT_DRAWINGS_OTHER_CURRENT,
AVG(CNT_DRAWINGS_POS_CURRENT) AS AVG_CNT_DRAWINGS_POS_CURRENT,
AVG(CNT_INSTALMENT_MATURE_CUM) AS AVG_CNT_INSTALMENT_MATURE_CUM
FROM
  credit_card_balance
GROUP BY
  SK_ID_CURR
""")

In [99]:
df_credit_card_balance_num.show()

+----------+------------------+--------------------+---------------------------+----------------------------+------------------------+------------------------------+----------------------------+---------------------------+-----------------------+-----------------------------+----------------------------+--------------------+------------------------+----------------------------+------------------------+------------------------------+----------------------------+-----------------------------+------------------+---------------+---------------------------+----------------------------+------------------------+------------------------------+----------------------------+---------------------------+-----------------------+-----------------------------+----------------------------+-----------------+------------------------+----------------------------+------------------------+------------------------------+----------------------------+-----------------------------+------------------+-----------

# **3º DF: Criando as variáveis numéricas e atrelando a diferentes status da variável categórica**

In [102]:
# Criando as agregações

agg = ['SUM', 'MIN', 'MAX', 'AVG']

# Criando a variável categórica

cat = ['NAME_CONTRACT_STATUS']

# Criando os domínios

dominios_cat = ['Approved', 'Refused', 'Sent proposal', 'Demand', 'Signed', 'Completed', 'Active']

# Criando variável numérica

var_num = ['MONTHS_BALANCE', 'AMT_BALANCE', 'AMT_CREDIT_LIMIT_ACTUAL', \
           'AMT_DRAWINGS_ATM_CURRENT', 'AMT_DRAWINGS_CURRENT', \
           'AMT_DRAWINGS_OTHER_CURRENT', 'AMT_DRAWINGS_POS_CURRENT', 'AMT_INST_MIN_REGULARITY',
           'AMT_PAYMENT_CURRENT', 'AMT_PAYMENT_TOTAL_CURRENT', 'AMT_RECEIVABLE_PRINCIPAL', \
           'AMT_RECIVABLE', 'AMT_TOTAL_RECEIVABLE', 'CNT_DRAWINGS_ATM_CURRENT', 'CNT_DRAWINGS_CURRENT', \
           'CNT_DRAWINGS_OTHER_CURRENT', 'CNT_DRAWINGS_POS_CURRENT', 'CNT_INSTALMENT_MATURE_CUM']

In [103]:
# Criando uma rotina que gera o texto da função de agregação

for ag in agg:
  for cate in cat:
    for dominio in dominios_cat:
      for var_n in var_num:
        print(ag +'(CASE WHEN ' + cate + '= ' + "'" + dominio + "'" + " THEN " + var_n + " ELSE 0 END) AS " + ag + "_" + cate + "_" + dominio.replace(" ", "_") + "_" + var_n + ',')

SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN MONTHS_BALANCE ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Approved_MONTHS_BALANCE,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN AMT_BALANCE ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Approved_AMT_BALANCE,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN AMT_CREDIT_LIMIT_ACTUAL ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Approved_AMT_CREDIT_LIMIT_ACTUAL,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN AMT_DRAWINGS_ATM_CURRENT ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Approved_AMT_DRAWINGS_ATM_CURRENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN AMT_DRAWINGS_CURRENT ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Approved_AMT_DRAWINGS_CURRENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN AMT_DRAWINGS_OTHER_CURRENT ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Approved_AMT_DRAWINGS_OTHER_CURRENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN AMT_DRAWINGS_POS_CURRENT ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Approved_AMT_DRAWINGS_P

In [104]:
df_credit_card_balance_and_name_contract_status_num = spark.sql("""
SELECT
  SK_ID_CURR,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN MONTHS_BALANCE ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Approved_MONTHS_BALANCE,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN AMT_BALANCE ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Approved_AMT_BALANCE,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN AMT_CREDIT_LIMIT_ACTUAL ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Approved_AMT_CREDIT_LIMIT_ACTUAL,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN AMT_DRAWINGS_ATM_CURRENT ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Approved_AMT_DRAWINGS_ATM_CURRENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN AMT_DRAWINGS_CURRENT ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Approved_AMT_DRAWINGS_CURRENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN AMT_DRAWINGS_OTHER_CURRENT ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Approved_AMT_DRAWINGS_OTHER_CURRENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN AMT_DRAWINGS_POS_CURRENT ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Approved_AMT_DRAWINGS_POS_CURRENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN AMT_INST_MIN_REGULARITY ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Approved_AMT_INST_MIN_REGULARITY,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN AMT_PAYMENT_CURRENT ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Approved_AMT_PAYMENT_CURRENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN AMT_PAYMENT_TOTAL_CURRENT ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Approved_AMT_PAYMENT_TOTAL_CURRENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN AMT_RECEIVABLE_PRINCIPAL ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Approved_AMT_RECEIVABLE_PRINCIPAL,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN AMT_RECIVABLE ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Approved_AMT_RECIVABLE,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN AMT_TOTAL_RECEIVABLE ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Approved_AMT_TOTAL_RECEIVABLE,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN CNT_DRAWINGS_ATM_CURRENT ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Approved_CNT_DRAWINGS_ATM_CURRENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN CNT_DRAWINGS_CURRENT ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Approved_CNT_DRAWINGS_CURRENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN CNT_DRAWINGS_OTHER_CURRENT ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Approved_CNT_DRAWINGS_OTHER_CURRENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN CNT_DRAWINGS_POS_CURRENT ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Approved_CNT_DRAWINGS_POS_CURRENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN CNT_INSTALMENT_MATURE_CUM ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Approved_CNT_INSTALMENT_MATURE_CUM,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN MONTHS_BALANCE ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Refused_MONTHS_BALANCE,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN AMT_BALANCE ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Refused_AMT_BALANCE,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN AMT_CREDIT_LIMIT_ACTUAL ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Refused_AMT_CREDIT_LIMIT_ACTUAL,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN AMT_DRAWINGS_ATM_CURRENT ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Refused_AMT_DRAWINGS_ATM_CURRENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN AMT_DRAWINGS_CURRENT ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Refused_AMT_DRAWINGS_CURRENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN AMT_DRAWINGS_OTHER_CURRENT ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Refused_AMT_DRAWINGS_OTHER_CURRENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN AMT_DRAWINGS_POS_CURRENT ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Refused_AMT_DRAWINGS_POS_CURRENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN AMT_INST_MIN_REGULARITY ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Refused_AMT_INST_MIN_REGULARITY,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN AMT_PAYMENT_CURRENT ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Refused_AMT_PAYMENT_CURRENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN AMT_PAYMENT_TOTAL_CURRENT ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Refused_AMT_PAYMENT_TOTAL_CURRENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN AMT_RECEIVABLE_PRINCIPAL ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Refused_AMT_RECEIVABLE_PRINCIPAL,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN AMT_RECIVABLE ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Refused_AMT_RECIVABLE,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN AMT_TOTAL_RECEIVABLE ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Refused_AMT_TOTAL_RECEIVABLE,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN CNT_DRAWINGS_ATM_CURRENT ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Refused_CNT_DRAWINGS_ATM_CURRENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN CNT_DRAWINGS_CURRENT ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Refused_CNT_DRAWINGS_CURRENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN CNT_DRAWINGS_OTHER_CURRENT ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Refused_CNT_DRAWINGS_OTHER_CURRENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN CNT_DRAWINGS_POS_CURRENT ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Refused_CNT_DRAWINGS_POS_CURRENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN CNT_INSTALMENT_MATURE_CUM ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Refused_CNT_INSTALMENT_MATURE_CUM,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Sent proposal' THEN MONTHS_BALANCE ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Sent_proposal_MONTHS_BALANCE,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Sent proposal' THEN AMT_BALANCE ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Sent_proposal_AMT_BALANCE,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Sent proposal' THEN AMT_CREDIT_LIMIT_ACTUAL ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Sent_proposal_AMT_CREDIT_LIMIT_ACTUAL,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Sent proposal' THEN AMT_DRAWINGS_ATM_CURRENT ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Sent_proposal_AMT_DRAWINGS_ATM_CURRENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Sent proposal' THEN AMT_DRAWINGS_CURRENT ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Sent_proposal_AMT_DRAWINGS_CURRENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Sent proposal' THEN AMT_DRAWINGS_OTHER_CURRENT ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Sent_proposal_AMT_DRAWINGS_OTHER_CURRENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Sent proposal' THEN AMT_DRAWINGS_POS_CURRENT ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Sent_proposal_AMT_DRAWINGS_POS_CURRENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Sent proposal' THEN AMT_INST_MIN_REGULARITY ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Sent_proposal_AMT_INST_MIN_REGULARITY,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Sent proposal' THEN AMT_PAYMENT_CURRENT ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Sent_proposal_AMT_PAYMENT_CURRENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Sent proposal' THEN AMT_PAYMENT_TOTAL_CURRENT ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Sent_proposal_AMT_PAYMENT_TOTAL_CURRENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Sent proposal' THEN AMT_RECEIVABLE_PRINCIPAL ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Sent_proposal_AMT_RECEIVABLE_PRINCIPAL,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Sent proposal' THEN AMT_RECIVABLE ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Sent_proposal_AMT_RECIVABLE,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Sent proposal' THEN AMT_TOTAL_RECEIVABLE ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Sent_proposal_AMT_TOTAL_RECEIVABLE,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Sent proposal' THEN CNT_DRAWINGS_ATM_CURRENT ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Sent_proposal_CNT_DRAWINGS_ATM_CURRENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Sent proposal' THEN CNT_DRAWINGS_CURRENT ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Sent_proposal_CNT_DRAWINGS_CURRENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Sent proposal' THEN CNT_DRAWINGS_OTHER_CURRENT ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Sent_proposal_CNT_DRAWINGS_OTHER_CURRENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Sent proposal' THEN CNT_DRAWINGS_POS_CURRENT ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Sent_proposal_CNT_DRAWINGS_POS_CURRENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Sent proposal' THEN CNT_INSTALMENT_MATURE_CUM ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Sent_proposal_CNT_INSTALMENT_MATURE_CUM,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Demand' THEN MONTHS_BALANCE ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Demand_MONTHS_BALANCE,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Demand' THEN AMT_BALANCE ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Demand_AMT_BALANCE,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Demand' THEN AMT_CREDIT_LIMIT_ACTUAL ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Demand_AMT_CREDIT_LIMIT_ACTUAL,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Demand' THEN AMT_DRAWINGS_ATM_CURRENT ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Demand_AMT_DRAWINGS_ATM_CURRENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Demand' THEN AMT_DRAWINGS_CURRENT ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Demand_AMT_DRAWINGS_CURRENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Demand' THEN AMT_DRAWINGS_OTHER_CURRENT ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Demand_AMT_DRAWINGS_OTHER_CURRENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Demand' THEN AMT_DRAWINGS_POS_CURRENT ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Demand_AMT_DRAWINGS_POS_CURRENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Demand' THEN AMT_INST_MIN_REGULARITY ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Demand_AMT_INST_MIN_REGULARITY,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Demand' THEN AMT_PAYMENT_CURRENT ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Demand_AMT_PAYMENT_CURRENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Demand' THEN AMT_PAYMENT_TOTAL_CURRENT ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Demand_AMT_PAYMENT_TOTAL_CURRENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Demand' THEN AMT_RECEIVABLE_PRINCIPAL ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Demand_AMT_RECEIVABLE_PRINCIPAL,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Demand' THEN AMT_RECIVABLE ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Demand_AMT_RECIVABLE,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Demand' THEN AMT_TOTAL_RECEIVABLE ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Demand_AMT_TOTAL_RECEIVABLE,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Demand' THEN CNT_DRAWINGS_ATM_CURRENT ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Demand_CNT_DRAWINGS_ATM_CURRENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Demand' THEN CNT_DRAWINGS_CURRENT ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Demand_CNT_DRAWINGS_CURRENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Demand' THEN CNT_DRAWINGS_OTHER_CURRENT ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Demand_CNT_DRAWINGS_OTHER_CURRENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Demand' THEN CNT_DRAWINGS_POS_CURRENT ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Demand_CNT_DRAWINGS_POS_CURRENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Demand' THEN CNT_INSTALMENT_MATURE_CUM ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Demand_CNT_INSTALMENT_MATURE_CUM,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Signed' THEN MONTHS_BALANCE ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Signed_MONTHS_BALANCE,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Signed' THEN AMT_BALANCE ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Signed_AMT_BALANCE,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Signed' THEN AMT_CREDIT_LIMIT_ACTUAL ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Signed_AMT_CREDIT_LIMIT_ACTUAL,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Signed' THEN AMT_DRAWINGS_ATM_CURRENT ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Signed_AMT_DRAWINGS_ATM_CURRENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Signed' THEN AMT_DRAWINGS_CURRENT ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Signed_AMT_DRAWINGS_CURRENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Signed' THEN AMT_DRAWINGS_OTHER_CURRENT ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Signed_AMT_DRAWINGS_OTHER_CURRENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Signed' THEN AMT_DRAWINGS_POS_CURRENT ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Signed_AMT_DRAWINGS_POS_CURRENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Signed' THEN AMT_INST_MIN_REGULARITY ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Signed_AMT_INST_MIN_REGULARITY,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Signed' THEN AMT_PAYMENT_CURRENT ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Signed_AMT_PAYMENT_CURRENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Signed' THEN AMT_PAYMENT_TOTAL_CURRENT ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Signed_AMT_PAYMENT_TOTAL_CURRENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Signed' THEN AMT_RECEIVABLE_PRINCIPAL ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Signed_AMT_RECEIVABLE_PRINCIPAL,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Signed' THEN AMT_RECIVABLE ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Signed_AMT_RECIVABLE,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Signed' THEN AMT_TOTAL_RECEIVABLE ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Signed_AMT_TOTAL_RECEIVABLE,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Signed' THEN CNT_DRAWINGS_ATM_CURRENT ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Signed_CNT_DRAWINGS_ATM_CURRENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Signed' THEN CNT_DRAWINGS_CURRENT ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Signed_CNT_DRAWINGS_CURRENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Signed' THEN CNT_DRAWINGS_OTHER_CURRENT ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Signed_CNT_DRAWINGS_OTHER_CURRENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Signed' THEN CNT_DRAWINGS_POS_CURRENT ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Signed_CNT_DRAWINGS_POS_CURRENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Signed' THEN CNT_INSTALMENT_MATURE_CUM ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Signed_CNT_INSTALMENT_MATURE_CUM,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Completed' THEN MONTHS_BALANCE ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Completed_MONTHS_BALANCE,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Completed' THEN AMT_BALANCE ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Completed_AMT_BALANCE,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Completed' THEN AMT_CREDIT_LIMIT_ACTUAL ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Completed_AMT_CREDIT_LIMIT_ACTUAL,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Completed' THEN AMT_DRAWINGS_ATM_CURRENT ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Completed_AMT_DRAWINGS_ATM_CURRENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Completed' THEN AMT_DRAWINGS_CURRENT ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Completed_AMT_DRAWINGS_CURRENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Completed' THEN AMT_DRAWINGS_OTHER_CURRENT ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Completed_AMT_DRAWINGS_OTHER_CURRENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Completed' THEN AMT_DRAWINGS_POS_CURRENT ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Completed_AMT_DRAWINGS_POS_CURRENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Completed' THEN AMT_INST_MIN_REGULARITY ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Completed_AMT_INST_MIN_REGULARITY,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Completed' THEN AMT_PAYMENT_CURRENT ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Completed_AMT_PAYMENT_CURRENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Completed' THEN AMT_PAYMENT_TOTAL_CURRENT ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Completed_AMT_PAYMENT_TOTAL_CURRENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Completed' THEN AMT_RECEIVABLE_PRINCIPAL ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Completed_AMT_RECEIVABLE_PRINCIPAL,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Completed' THEN AMT_RECIVABLE ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Completed_AMT_RECIVABLE,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Completed' THEN AMT_TOTAL_RECEIVABLE ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Completed_AMT_TOTAL_RECEIVABLE,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Completed' THEN CNT_DRAWINGS_ATM_CURRENT ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Completed_CNT_DRAWINGS_ATM_CURRENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Completed' THEN CNT_DRAWINGS_CURRENT ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Completed_CNT_DRAWINGS_CURRENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Completed' THEN CNT_DRAWINGS_OTHER_CURRENT ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Completed_CNT_DRAWINGS_OTHER_CURRENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Completed' THEN CNT_DRAWINGS_POS_CURRENT ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Completed_CNT_DRAWINGS_POS_CURRENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Completed' THEN CNT_INSTALMENT_MATURE_CUM ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Completed_CNT_INSTALMENT_MATURE_CUM,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Active' THEN MONTHS_BALANCE ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Active_MONTHS_BALANCE,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Active' THEN AMT_BALANCE ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Active_AMT_BALANCE,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Active' THEN AMT_CREDIT_LIMIT_ACTUAL ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Active_AMT_CREDIT_LIMIT_ACTUAL,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Active' THEN AMT_DRAWINGS_ATM_CURRENT ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Active_AMT_DRAWINGS_ATM_CURRENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Active' THEN AMT_DRAWINGS_CURRENT ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Active_AMT_DRAWINGS_CURRENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Active' THEN AMT_DRAWINGS_OTHER_CURRENT ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Active_AMT_DRAWINGS_OTHER_CURRENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Active' THEN AMT_DRAWINGS_POS_CURRENT ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Active_AMT_DRAWINGS_POS_CURRENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Active' THEN AMT_INST_MIN_REGULARITY ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Active_AMT_INST_MIN_REGULARITY,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Active' THEN AMT_PAYMENT_CURRENT ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Active_AMT_PAYMENT_CURRENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Active' THEN AMT_PAYMENT_TOTAL_CURRENT ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Active_AMT_PAYMENT_TOTAL_CURRENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Active' THEN AMT_RECEIVABLE_PRINCIPAL ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Active_AMT_RECEIVABLE_PRINCIPAL,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Active' THEN AMT_RECIVABLE ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Active_AMT_RECIVABLE,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Active' THEN AMT_TOTAL_RECEIVABLE ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Active_AMT_TOTAL_RECEIVABLE,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Active' THEN CNT_DRAWINGS_ATM_CURRENT ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Active_CNT_DRAWINGS_ATM_CURRENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Active' THEN CNT_DRAWINGS_CURRENT ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Active_CNT_DRAWINGS_CURRENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Active' THEN CNT_DRAWINGS_OTHER_CURRENT ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Active_CNT_DRAWINGS_OTHER_CURRENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Active' THEN CNT_DRAWINGS_POS_CURRENT ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Active_CNT_DRAWINGS_POS_CURRENT,
SUM(CASE WHEN NAME_CONTRACT_STATUS= 'Active' THEN CNT_INSTALMENT_MATURE_CUM ELSE 0 END) AS SUM_NAME_CONTRACT_STATUS_Active_CNT_INSTALMENT_MATURE_CUM,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN MONTHS_BALANCE ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Approved_MONTHS_BALANCE,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN AMT_BALANCE ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Approved_AMT_BALANCE,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN AMT_CREDIT_LIMIT_ACTUAL ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Approved_AMT_CREDIT_LIMIT_ACTUAL,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN AMT_DRAWINGS_ATM_CURRENT ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Approved_AMT_DRAWINGS_ATM_CURRENT,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN AMT_DRAWINGS_CURRENT ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Approved_AMT_DRAWINGS_CURRENT,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN AMT_DRAWINGS_OTHER_CURRENT ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Approved_AMT_DRAWINGS_OTHER_CURRENT,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN AMT_DRAWINGS_POS_CURRENT ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Approved_AMT_DRAWINGS_POS_CURRENT,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN AMT_INST_MIN_REGULARITY ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Approved_AMT_INST_MIN_REGULARITY,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN AMT_PAYMENT_CURRENT ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Approved_AMT_PAYMENT_CURRENT,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN AMT_PAYMENT_TOTAL_CURRENT ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Approved_AMT_PAYMENT_TOTAL_CURRENT,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN AMT_RECEIVABLE_PRINCIPAL ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Approved_AMT_RECEIVABLE_PRINCIPAL,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN AMT_RECIVABLE ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Approved_AMT_RECIVABLE,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN AMT_TOTAL_RECEIVABLE ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Approved_AMT_TOTAL_RECEIVABLE,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN CNT_DRAWINGS_ATM_CURRENT ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Approved_CNT_DRAWINGS_ATM_CURRENT,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN CNT_DRAWINGS_CURRENT ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Approved_CNT_DRAWINGS_CURRENT,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN CNT_DRAWINGS_OTHER_CURRENT ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Approved_CNT_DRAWINGS_OTHER_CURRENT,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN CNT_DRAWINGS_POS_CURRENT ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Approved_CNT_DRAWINGS_POS_CURRENT,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN CNT_INSTALMENT_MATURE_CUM ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Approved_CNT_INSTALMENT_MATURE_CUM,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN MONTHS_BALANCE ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Refused_MONTHS_BALANCE,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN AMT_BALANCE ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Refused_AMT_BALANCE,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN AMT_CREDIT_LIMIT_ACTUAL ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Refused_AMT_CREDIT_LIMIT_ACTUAL,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN AMT_DRAWINGS_ATM_CURRENT ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Refused_AMT_DRAWINGS_ATM_CURRENT,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN AMT_DRAWINGS_CURRENT ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Refused_AMT_DRAWINGS_CURRENT,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN AMT_DRAWINGS_OTHER_CURRENT ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Refused_AMT_DRAWINGS_OTHER_CURRENT,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN AMT_DRAWINGS_POS_CURRENT ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Refused_AMT_DRAWINGS_POS_CURRENT,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN AMT_INST_MIN_REGULARITY ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Refused_AMT_INST_MIN_REGULARITY,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN AMT_PAYMENT_CURRENT ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Refused_AMT_PAYMENT_CURRENT,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN AMT_PAYMENT_TOTAL_CURRENT ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Refused_AMT_PAYMENT_TOTAL_CURRENT,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN AMT_RECEIVABLE_PRINCIPAL ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Refused_AMT_RECEIVABLE_PRINCIPAL,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN AMT_RECIVABLE ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Refused_AMT_RECIVABLE,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN AMT_TOTAL_RECEIVABLE ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Refused_AMT_TOTAL_RECEIVABLE,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN CNT_DRAWINGS_ATM_CURRENT ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Refused_CNT_DRAWINGS_ATM_CURRENT,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN CNT_DRAWINGS_CURRENT ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Refused_CNT_DRAWINGS_CURRENT,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN CNT_DRAWINGS_OTHER_CURRENT ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Refused_CNT_DRAWINGS_OTHER_CURRENT,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN CNT_DRAWINGS_POS_CURRENT ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Refused_CNT_DRAWINGS_POS_CURRENT,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN CNT_INSTALMENT_MATURE_CUM ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Refused_CNT_INSTALMENT_MATURE_CUM,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Sent proposal' THEN MONTHS_BALANCE ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Sent_proposal_MONTHS_BALANCE,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Sent proposal' THEN AMT_BALANCE ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Sent_proposal_AMT_BALANCE,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Sent proposal' THEN AMT_CREDIT_LIMIT_ACTUAL ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Sent_proposal_AMT_CREDIT_LIMIT_ACTUAL,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Sent proposal' THEN AMT_DRAWINGS_ATM_CURRENT ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Sent_proposal_AMT_DRAWINGS_ATM_CURRENT,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Sent proposal' THEN AMT_DRAWINGS_CURRENT ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Sent_proposal_AMT_DRAWINGS_CURRENT,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Sent proposal' THEN AMT_DRAWINGS_OTHER_CURRENT ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Sent_proposal_AMT_DRAWINGS_OTHER_CURRENT,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Sent proposal' THEN AMT_DRAWINGS_POS_CURRENT ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Sent_proposal_AMT_DRAWINGS_POS_CURRENT,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Sent proposal' THEN AMT_INST_MIN_REGULARITY ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Sent_proposal_AMT_INST_MIN_REGULARITY,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Sent proposal' THEN AMT_PAYMENT_CURRENT ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Sent_proposal_AMT_PAYMENT_CURRENT,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Sent proposal' THEN AMT_PAYMENT_TOTAL_CURRENT ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Sent_proposal_AMT_PAYMENT_TOTAL_CURRENT,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Sent proposal' THEN AMT_RECEIVABLE_PRINCIPAL ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Sent_proposal_AMT_RECEIVABLE_PRINCIPAL,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Sent proposal' THEN AMT_RECIVABLE ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Sent_proposal_AMT_RECIVABLE,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Sent proposal' THEN AMT_TOTAL_RECEIVABLE ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Sent_proposal_AMT_TOTAL_RECEIVABLE,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Sent proposal' THEN CNT_DRAWINGS_ATM_CURRENT ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Sent_proposal_CNT_DRAWINGS_ATM_CURRENT,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Sent proposal' THEN CNT_DRAWINGS_CURRENT ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Sent_proposal_CNT_DRAWINGS_CURRENT,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Sent proposal' THEN CNT_DRAWINGS_OTHER_CURRENT ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Sent_proposal_CNT_DRAWINGS_OTHER_CURRENT,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Sent proposal' THEN CNT_DRAWINGS_POS_CURRENT ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Sent_proposal_CNT_DRAWINGS_POS_CURRENT,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Sent proposal' THEN CNT_INSTALMENT_MATURE_CUM ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Sent_proposal_CNT_INSTALMENT_MATURE_CUM,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Demand' THEN MONTHS_BALANCE ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Demand_MONTHS_BALANCE,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Demand' THEN AMT_BALANCE ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Demand_AMT_BALANCE,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Demand' THEN AMT_CREDIT_LIMIT_ACTUAL ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Demand_AMT_CREDIT_LIMIT_ACTUAL,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Demand' THEN AMT_DRAWINGS_ATM_CURRENT ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Demand_AMT_DRAWINGS_ATM_CURRENT,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Demand' THEN AMT_DRAWINGS_CURRENT ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Demand_AMT_DRAWINGS_CURRENT,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Demand' THEN AMT_DRAWINGS_OTHER_CURRENT ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Demand_AMT_DRAWINGS_OTHER_CURRENT,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Demand' THEN AMT_DRAWINGS_POS_CURRENT ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Demand_AMT_DRAWINGS_POS_CURRENT,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Demand' THEN AMT_INST_MIN_REGULARITY ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Demand_AMT_INST_MIN_REGULARITY,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Demand' THEN AMT_PAYMENT_CURRENT ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Demand_AMT_PAYMENT_CURRENT,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Demand' THEN AMT_PAYMENT_TOTAL_CURRENT ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Demand_AMT_PAYMENT_TOTAL_CURRENT,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Demand' THEN AMT_RECEIVABLE_PRINCIPAL ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Demand_AMT_RECEIVABLE_PRINCIPAL,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Demand' THEN AMT_RECIVABLE ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Demand_AMT_RECIVABLE,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Demand' THEN AMT_TOTAL_RECEIVABLE ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Demand_AMT_TOTAL_RECEIVABLE,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Demand' THEN CNT_DRAWINGS_ATM_CURRENT ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Demand_CNT_DRAWINGS_ATM_CURRENT,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Demand' THEN CNT_DRAWINGS_CURRENT ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Demand_CNT_DRAWINGS_CURRENT,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Demand' THEN CNT_DRAWINGS_OTHER_CURRENT ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Demand_CNT_DRAWINGS_OTHER_CURRENT,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Demand' THEN CNT_DRAWINGS_POS_CURRENT ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Demand_CNT_DRAWINGS_POS_CURRENT,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Demand' THEN CNT_INSTALMENT_MATURE_CUM ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Demand_CNT_INSTALMENT_MATURE_CUM,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Signed' THEN MONTHS_BALANCE ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Signed_MONTHS_BALANCE,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Signed' THEN AMT_BALANCE ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Signed_AMT_BALANCE,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Signed' THEN AMT_CREDIT_LIMIT_ACTUAL ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Signed_AMT_CREDIT_LIMIT_ACTUAL,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Signed' THEN AMT_DRAWINGS_ATM_CURRENT ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Signed_AMT_DRAWINGS_ATM_CURRENT,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Signed' THEN AMT_DRAWINGS_CURRENT ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Signed_AMT_DRAWINGS_CURRENT,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Signed' THEN AMT_DRAWINGS_OTHER_CURRENT ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Signed_AMT_DRAWINGS_OTHER_CURRENT,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Signed' THEN AMT_DRAWINGS_POS_CURRENT ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Signed_AMT_DRAWINGS_POS_CURRENT,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Signed' THEN AMT_INST_MIN_REGULARITY ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Signed_AMT_INST_MIN_REGULARITY,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Signed' THEN AMT_PAYMENT_CURRENT ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Signed_AMT_PAYMENT_CURRENT,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Signed' THEN AMT_PAYMENT_TOTAL_CURRENT ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Signed_AMT_PAYMENT_TOTAL_CURRENT,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Signed' THEN AMT_RECEIVABLE_PRINCIPAL ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Signed_AMT_RECEIVABLE_PRINCIPAL,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Signed' THEN AMT_RECIVABLE ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Signed_AMT_RECIVABLE,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Signed' THEN AMT_TOTAL_RECEIVABLE ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Signed_AMT_TOTAL_RECEIVABLE,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Signed' THEN CNT_DRAWINGS_ATM_CURRENT ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Signed_CNT_DRAWINGS_ATM_CURRENT,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Signed' THEN CNT_DRAWINGS_CURRENT ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Signed_CNT_DRAWINGS_CURRENT,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Signed' THEN CNT_DRAWINGS_OTHER_CURRENT ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Signed_CNT_DRAWINGS_OTHER_CURRENT,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Signed' THEN CNT_DRAWINGS_POS_CURRENT ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Signed_CNT_DRAWINGS_POS_CURRENT,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Signed' THEN CNT_INSTALMENT_MATURE_CUM ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Signed_CNT_INSTALMENT_MATURE_CUM,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Completed' THEN MONTHS_BALANCE ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Completed_MONTHS_BALANCE,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Completed' THEN AMT_BALANCE ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Completed_AMT_BALANCE,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Completed' THEN AMT_CREDIT_LIMIT_ACTUAL ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Completed_AMT_CREDIT_LIMIT_ACTUAL,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Completed' THEN AMT_DRAWINGS_ATM_CURRENT ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Completed_AMT_DRAWINGS_ATM_CURRENT,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Completed' THEN AMT_DRAWINGS_CURRENT ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Completed_AMT_DRAWINGS_CURRENT,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Completed' THEN AMT_DRAWINGS_OTHER_CURRENT ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Completed_AMT_DRAWINGS_OTHER_CURRENT,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Completed' THEN AMT_DRAWINGS_POS_CURRENT ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Completed_AMT_DRAWINGS_POS_CURRENT,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Completed' THEN AMT_INST_MIN_REGULARITY ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Completed_AMT_INST_MIN_REGULARITY,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Completed' THEN AMT_PAYMENT_CURRENT ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Completed_AMT_PAYMENT_CURRENT,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Completed' THEN AMT_PAYMENT_TOTAL_CURRENT ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Completed_AMT_PAYMENT_TOTAL_CURRENT,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Completed' THEN AMT_RECEIVABLE_PRINCIPAL ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Completed_AMT_RECEIVABLE_PRINCIPAL,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Completed' THEN AMT_RECIVABLE ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Completed_AMT_RECIVABLE,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Completed' THEN AMT_TOTAL_RECEIVABLE ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Completed_AMT_TOTAL_RECEIVABLE,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Completed' THEN CNT_DRAWINGS_ATM_CURRENT ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Completed_CNT_DRAWINGS_ATM_CURRENT,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Completed' THEN CNT_DRAWINGS_CURRENT ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Completed_CNT_DRAWINGS_CURRENT,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Completed' THEN CNT_DRAWINGS_OTHER_CURRENT ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Completed_CNT_DRAWINGS_OTHER_CURRENT,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Completed' THEN CNT_DRAWINGS_POS_CURRENT ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Completed_CNT_DRAWINGS_POS_CURRENT,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Completed' THEN CNT_INSTALMENT_MATURE_CUM ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Completed_CNT_INSTALMENT_MATURE_CUM,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Active' THEN MONTHS_BALANCE ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Active_MONTHS_BALANCE,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Active' THEN AMT_BALANCE ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Active_AMT_BALANCE,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Active' THEN AMT_CREDIT_LIMIT_ACTUAL ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Active_AMT_CREDIT_LIMIT_ACTUAL,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Active' THEN AMT_DRAWINGS_ATM_CURRENT ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Active_AMT_DRAWINGS_ATM_CURRENT,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Active' THEN AMT_DRAWINGS_CURRENT ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Active_AMT_DRAWINGS_CURRENT,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Active' THEN AMT_DRAWINGS_OTHER_CURRENT ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Active_AMT_DRAWINGS_OTHER_CURRENT,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Active' THEN AMT_DRAWINGS_POS_CURRENT ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Active_AMT_DRAWINGS_POS_CURRENT,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Active' THEN AMT_INST_MIN_REGULARITY ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Active_AMT_INST_MIN_REGULARITY,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Active' THEN AMT_PAYMENT_CURRENT ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Active_AMT_PAYMENT_CURRENT,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Active' THEN AMT_PAYMENT_TOTAL_CURRENT ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Active_AMT_PAYMENT_TOTAL_CURRENT,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Active' THEN AMT_RECEIVABLE_PRINCIPAL ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Active_AMT_RECEIVABLE_PRINCIPAL,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Active' THEN AMT_RECIVABLE ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Active_AMT_RECIVABLE,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Active' THEN AMT_TOTAL_RECEIVABLE ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Active_AMT_TOTAL_RECEIVABLE,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Active' THEN CNT_DRAWINGS_ATM_CURRENT ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Active_CNT_DRAWINGS_ATM_CURRENT,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Active' THEN CNT_DRAWINGS_CURRENT ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Active_CNT_DRAWINGS_CURRENT,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Active' THEN CNT_DRAWINGS_OTHER_CURRENT ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Active_CNT_DRAWINGS_OTHER_CURRENT,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Active' THEN CNT_DRAWINGS_POS_CURRENT ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Active_CNT_DRAWINGS_POS_CURRENT,
MIN(CASE WHEN NAME_CONTRACT_STATUS= 'Active' THEN CNT_INSTALMENT_MATURE_CUM ELSE 0 END) AS MIN_NAME_CONTRACT_STATUS_Active_CNT_INSTALMENT_MATURE_CUM,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN MONTHS_BALANCE ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Approved_MONTHS_BALANCE,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN AMT_BALANCE ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Approved_AMT_BALANCE,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN AMT_CREDIT_LIMIT_ACTUAL ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Approved_AMT_CREDIT_LIMIT_ACTUAL,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN AMT_DRAWINGS_ATM_CURRENT ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Approved_AMT_DRAWINGS_ATM_CURRENT,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN AMT_DRAWINGS_CURRENT ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Approved_AMT_DRAWINGS_CURRENT,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN AMT_DRAWINGS_OTHER_CURRENT ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Approved_AMT_DRAWINGS_OTHER_CURRENT,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN AMT_DRAWINGS_POS_CURRENT ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Approved_AMT_DRAWINGS_POS_CURRENT,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN AMT_INST_MIN_REGULARITY ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Approved_AMT_INST_MIN_REGULARITY,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN AMT_PAYMENT_CURRENT ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Approved_AMT_PAYMENT_CURRENT,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN AMT_PAYMENT_TOTAL_CURRENT ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Approved_AMT_PAYMENT_TOTAL_CURRENT,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN AMT_RECEIVABLE_PRINCIPAL ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Approved_AMT_RECEIVABLE_PRINCIPAL,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN AMT_RECIVABLE ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Approved_AMT_RECIVABLE,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN AMT_TOTAL_RECEIVABLE ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Approved_AMT_TOTAL_RECEIVABLE,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN CNT_DRAWINGS_ATM_CURRENT ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Approved_CNT_DRAWINGS_ATM_CURRENT,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN CNT_DRAWINGS_CURRENT ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Approved_CNT_DRAWINGS_CURRENT,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN CNT_DRAWINGS_OTHER_CURRENT ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Approved_CNT_DRAWINGS_OTHER_CURRENT,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN CNT_DRAWINGS_POS_CURRENT ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Approved_CNT_DRAWINGS_POS_CURRENT,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN CNT_INSTALMENT_MATURE_CUM ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Approved_CNT_INSTALMENT_MATURE_CUM,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN MONTHS_BALANCE ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Refused_MONTHS_BALANCE,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN AMT_BALANCE ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Refused_AMT_BALANCE,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN AMT_CREDIT_LIMIT_ACTUAL ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Refused_AMT_CREDIT_LIMIT_ACTUAL,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN AMT_DRAWINGS_ATM_CURRENT ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Refused_AMT_DRAWINGS_ATM_CURRENT,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN AMT_DRAWINGS_CURRENT ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Refused_AMT_DRAWINGS_CURRENT,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN AMT_DRAWINGS_OTHER_CURRENT ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Refused_AMT_DRAWINGS_OTHER_CURRENT,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN AMT_DRAWINGS_POS_CURRENT ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Refused_AMT_DRAWINGS_POS_CURRENT,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN AMT_INST_MIN_REGULARITY ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Refused_AMT_INST_MIN_REGULARITY,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN AMT_PAYMENT_CURRENT ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Refused_AMT_PAYMENT_CURRENT,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN AMT_PAYMENT_TOTAL_CURRENT ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Refused_AMT_PAYMENT_TOTAL_CURRENT,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN AMT_RECEIVABLE_PRINCIPAL ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Refused_AMT_RECEIVABLE_PRINCIPAL,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN AMT_RECIVABLE ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Refused_AMT_RECIVABLE,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN AMT_TOTAL_RECEIVABLE ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Refused_AMT_TOTAL_RECEIVABLE,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN CNT_DRAWINGS_ATM_CURRENT ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Refused_CNT_DRAWINGS_ATM_CURRENT,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN CNT_DRAWINGS_CURRENT ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Refused_CNT_DRAWINGS_CURRENT,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN CNT_DRAWINGS_OTHER_CURRENT ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Refused_CNT_DRAWINGS_OTHER_CURRENT,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN CNT_DRAWINGS_POS_CURRENT ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Refused_CNT_DRAWINGS_POS_CURRENT,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN CNT_INSTALMENT_MATURE_CUM ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Refused_CNT_INSTALMENT_MATURE_CUM,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Sent proposal' THEN MONTHS_BALANCE ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Sent_proposal_MONTHS_BALANCE,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Sent proposal' THEN AMT_BALANCE ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Sent_proposal_AMT_BALANCE,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Sent proposal' THEN AMT_CREDIT_LIMIT_ACTUAL ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Sent_proposal_AMT_CREDIT_LIMIT_ACTUAL,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Sent proposal' THEN AMT_DRAWINGS_ATM_CURRENT ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Sent_proposal_AMT_DRAWINGS_ATM_CURRENT,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Sent proposal' THEN AMT_DRAWINGS_CURRENT ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Sent_proposal_AMT_DRAWINGS_CURRENT,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Sent proposal' THEN AMT_DRAWINGS_OTHER_CURRENT ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Sent_proposal_AMT_DRAWINGS_OTHER_CURRENT,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Sent proposal' THEN AMT_DRAWINGS_POS_CURRENT ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Sent_proposal_AMT_DRAWINGS_POS_CURRENT,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Sent proposal' THEN AMT_INST_MIN_REGULARITY ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Sent_proposal_AMT_INST_MIN_REGULARITY,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Sent proposal' THEN AMT_PAYMENT_CURRENT ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Sent_proposal_AMT_PAYMENT_CURRENT,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Sent proposal' THEN AMT_PAYMENT_TOTAL_CURRENT ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Sent_proposal_AMT_PAYMENT_TOTAL_CURRENT,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Sent proposal' THEN AMT_RECEIVABLE_PRINCIPAL ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Sent_proposal_AMT_RECEIVABLE_PRINCIPAL,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Sent proposal' THEN AMT_RECIVABLE ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Sent_proposal_AMT_RECIVABLE,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Sent proposal' THEN AMT_TOTAL_RECEIVABLE ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Sent_proposal_AMT_TOTAL_RECEIVABLE,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Sent proposal' THEN CNT_DRAWINGS_ATM_CURRENT ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Sent_proposal_CNT_DRAWINGS_ATM_CURRENT,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Sent proposal' THEN CNT_DRAWINGS_CURRENT ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Sent_proposal_CNT_DRAWINGS_CURRENT,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Sent proposal' THEN CNT_DRAWINGS_OTHER_CURRENT ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Sent_proposal_CNT_DRAWINGS_OTHER_CURRENT,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Sent proposal' THEN CNT_DRAWINGS_POS_CURRENT ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Sent_proposal_CNT_DRAWINGS_POS_CURRENT,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Sent proposal' THEN CNT_INSTALMENT_MATURE_CUM ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Sent_proposal_CNT_INSTALMENT_MATURE_CUM,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Demand' THEN MONTHS_BALANCE ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Demand_MONTHS_BALANCE,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Demand' THEN AMT_BALANCE ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Demand_AMT_BALANCE,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Demand' THEN AMT_CREDIT_LIMIT_ACTUAL ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Demand_AMT_CREDIT_LIMIT_ACTUAL,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Demand' THEN AMT_DRAWINGS_ATM_CURRENT ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Demand_AMT_DRAWINGS_ATM_CURRENT,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Demand' THEN AMT_DRAWINGS_CURRENT ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Demand_AMT_DRAWINGS_CURRENT,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Demand' THEN AMT_DRAWINGS_OTHER_CURRENT ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Demand_AMT_DRAWINGS_OTHER_CURRENT,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Demand' THEN AMT_DRAWINGS_POS_CURRENT ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Demand_AMT_DRAWINGS_POS_CURRENT,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Demand' THEN AMT_INST_MIN_REGULARITY ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Demand_AMT_INST_MIN_REGULARITY,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Demand' THEN AMT_PAYMENT_CURRENT ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Demand_AMT_PAYMENT_CURRENT,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Demand' THEN AMT_PAYMENT_TOTAL_CURRENT ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Demand_AMT_PAYMENT_TOTAL_CURRENT,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Demand' THEN AMT_RECEIVABLE_PRINCIPAL ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Demand_AMT_RECEIVABLE_PRINCIPAL,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Demand' THEN AMT_RECIVABLE ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Demand_AMT_RECIVABLE,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Demand' THEN AMT_TOTAL_RECEIVABLE ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Demand_AMT_TOTAL_RECEIVABLE,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Demand' THEN CNT_DRAWINGS_ATM_CURRENT ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Demand_CNT_DRAWINGS_ATM_CURRENT,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Demand' THEN CNT_DRAWINGS_CURRENT ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Demand_CNT_DRAWINGS_CURRENT,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Demand' THEN CNT_DRAWINGS_OTHER_CURRENT ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Demand_CNT_DRAWINGS_OTHER_CURRENT,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Demand' THEN CNT_DRAWINGS_POS_CURRENT ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Demand_CNT_DRAWINGS_POS_CURRENT,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Demand' THEN CNT_INSTALMENT_MATURE_CUM ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Demand_CNT_INSTALMENT_MATURE_CUM,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Signed' THEN MONTHS_BALANCE ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Signed_MONTHS_BALANCE,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Signed' THEN AMT_BALANCE ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Signed_AMT_BALANCE,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Signed' THEN AMT_CREDIT_LIMIT_ACTUAL ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Signed_AMT_CREDIT_LIMIT_ACTUAL,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Signed' THEN AMT_DRAWINGS_ATM_CURRENT ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Signed_AMT_DRAWINGS_ATM_CURRENT,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Signed' THEN AMT_DRAWINGS_CURRENT ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Signed_AMT_DRAWINGS_CURRENT,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Signed' THEN AMT_DRAWINGS_OTHER_CURRENT ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Signed_AMT_DRAWINGS_OTHER_CURRENT,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Signed' THEN AMT_DRAWINGS_POS_CURRENT ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Signed_AMT_DRAWINGS_POS_CURRENT,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Signed' THEN AMT_INST_MIN_REGULARITY ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Signed_AMT_INST_MIN_REGULARITY,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Signed' THEN AMT_PAYMENT_CURRENT ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Signed_AMT_PAYMENT_CURRENT,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Signed' THEN AMT_PAYMENT_TOTAL_CURRENT ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Signed_AMT_PAYMENT_TOTAL_CURRENT,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Signed' THEN AMT_RECEIVABLE_PRINCIPAL ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Signed_AMT_RECEIVABLE_PRINCIPAL,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Signed' THEN AMT_RECIVABLE ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Signed_AMT_RECIVABLE,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Signed' THEN AMT_TOTAL_RECEIVABLE ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Signed_AMT_TOTAL_RECEIVABLE,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Signed' THEN CNT_DRAWINGS_ATM_CURRENT ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Signed_CNT_DRAWINGS_ATM_CURRENT,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Signed' THEN CNT_DRAWINGS_CURRENT ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Signed_CNT_DRAWINGS_CURRENT,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Signed' THEN CNT_DRAWINGS_OTHER_CURRENT ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Signed_CNT_DRAWINGS_OTHER_CURRENT,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Signed' THEN CNT_DRAWINGS_POS_CURRENT ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Signed_CNT_DRAWINGS_POS_CURRENT,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Signed' THEN CNT_INSTALMENT_MATURE_CUM ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Signed_CNT_INSTALMENT_MATURE_CUM,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Completed' THEN MONTHS_BALANCE ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Completed_MONTHS_BALANCE,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Completed' THEN AMT_BALANCE ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Completed_AMT_BALANCE,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Completed' THEN AMT_CREDIT_LIMIT_ACTUAL ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Completed_AMT_CREDIT_LIMIT_ACTUAL,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Completed' THEN AMT_DRAWINGS_ATM_CURRENT ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Completed_AMT_DRAWINGS_ATM_CURRENT,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Completed' THEN AMT_DRAWINGS_CURRENT ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Completed_AMT_DRAWINGS_CURRENT,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Completed' THEN AMT_DRAWINGS_OTHER_CURRENT ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Completed_AMT_DRAWINGS_OTHER_CURRENT,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Completed' THEN AMT_DRAWINGS_POS_CURRENT ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Completed_AMT_DRAWINGS_POS_CURRENT,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Completed' THEN AMT_INST_MIN_REGULARITY ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Completed_AMT_INST_MIN_REGULARITY,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Completed' THEN AMT_PAYMENT_CURRENT ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Completed_AMT_PAYMENT_CURRENT,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Completed' THEN AMT_PAYMENT_TOTAL_CURRENT ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Completed_AMT_PAYMENT_TOTAL_CURRENT,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Completed' THEN AMT_RECEIVABLE_PRINCIPAL ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Completed_AMT_RECEIVABLE_PRINCIPAL,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Completed' THEN AMT_RECIVABLE ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Completed_AMT_RECIVABLE,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Completed' THEN AMT_TOTAL_RECEIVABLE ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Completed_AMT_TOTAL_RECEIVABLE,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Completed' THEN CNT_DRAWINGS_ATM_CURRENT ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Completed_CNT_DRAWINGS_ATM_CURRENT,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Completed' THEN CNT_DRAWINGS_CURRENT ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Completed_CNT_DRAWINGS_CURRENT,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Completed' THEN CNT_DRAWINGS_OTHER_CURRENT ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Completed_CNT_DRAWINGS_OTHER_CURRENT,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Completed' THEN CNT_DRAWINGS_POS_CURRENT ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Completed_CNT_DRAWINGS_POS_CURRENT,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Completed' THEN CNT_INSTALMENT_MATURE_CUM ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Completed_CNT_INSTALMENT_MATURE_CUM,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Active' THEN MONTHS_BALANCE ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Active_MONTHS_BALANCE,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Active' THEN AMT_BALANCE ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Active_AMT_BALANCE,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Active' THEN AMT_CREDIT_LIMIT_ACTUAL ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Active_AMT_CREDIT_LIMIT_ACTUAL,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Active' THEN AMT_DRAWINGS_ATM_CURRENT ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Active_AMT_DRAWINGS_ATM_CURRENT,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Active' THEN AMT_DRAWINGS_CURRENT ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Active_AMT_DRAWINGS_CURRENT,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Active' THEN AMT_DRAWINGS_OTHER_CURRENT ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Active_AMT_DRAWINGS_OTHER_CURRENT,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Active' THEN AMT_DRAWINGS_POS_CURRENT ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Active_AMT_DRAWINGS_POS_CURRENT,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Active' THEN AMT_INST_MIN_REGULARITY ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Active_AMT_INST_MIN_REGULARITY,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Active' THEN AMT_PAYMENT_CURRENT ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Active_AMT_PAYMENT_CURRENT,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Active' THEN AMT_PAYMENT_TOTAL_CURRENT ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Active_AMT_PAYMENT_TOTAL_CURRENT,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Active' THEN AMT_RECEIVABLE_PRINCIPAL ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Active_AMT_RECEIVABLE_PRINCIPAL,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Active' THEN AMT_RECIVABLE ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Active_AMT_RECIVABLE,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Active' THEN AMT_TOTAL_RECEIVABLE ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Active_AMT_TOTAL_RECEIVABLE,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Active' THEN CNT_DRAWINGS_ATM_CURRENT ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Active_CNT_DRAWINGS_ATM_CURRENT,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Active' THEN CNT_DRAWINGS_CURRENT ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Active_CNT_DRAWINGS_CURRENT,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Active' THEN CNT_DRAWINGS_OTHER_CURRENT ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Active_CNT_DRAWINGS_OTHER_CURRENT,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Active' THEN CNT_DRAWINGS_POS_CURRENT ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Active_CNT_DRAWINGS_POS_CURRENT,
MAX(CASE WHEN NAME_CONTRACT_STATUS= 'Active' THEN CNT_INSTALMENT_MATURE_CUM ELSE 0 END) AS MAX_NAME_CONTRACT_STATUS_Active_CNT_INSTALMENT_MATURE_CUM,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN MONTHS_BALANCE ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Approved_MONTHS_BALANCE,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN AMT_BALANCE ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Approved_AMT_BALANCE,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN AMT_CREDIT_LIMIT_ACTUAL ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Approved_AMT_CREDIT_LIMIT_ACTUAL,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN AMT_DRAWINGS_ATM_CURRENT ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Approved_AMT_DRAWINGS_ATM_CURRENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN AMT_DRAWINGS_CURRENT ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Approved_AMT_DRAWINGS_CURRENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN AMT_DRAWINGS_OTHER_CURRENT ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Approved_AMT_DRAWINGS_OTHER_CURRENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN AMT_DRAWINGS_POS_CURRENT ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Approved_AMT_DRAWINGS_POS_CURRENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN AMT_INST_MIN_REGULARITY ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Approved_AMT_INST_MIN_REGULARITY,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN AMT_PAYMENT_CURRENT ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Approved_AMT_PAYMENT_CURRENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN AMT_PAYMENT_TOTAL_CURRENT ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Approved_AMT_PAYMENT_TOTAL_CURRENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN AMT_RECEIVABLE_PRINCIPAL ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Approved_AMT_RECEIVABLE_PRINCIPAL,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN AMT_RECIVABLE ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Approved_AMT_RECIVABLE,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN AMT_TOTAL_RECEIVABLE ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Approved_AMT_TOTAL_RECEIVABLE,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN CNT_DRAWINGS_ATM_CURRENT ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Approved_CNT_DRAWINGS_ATM_CURRENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN CNT_DRAWINGS_CURRENT ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Approved_CNT_DRAWINGS_CURRENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN CNT_DRAWINGS_OTHER_CURRENT ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Approved_CNT_DRAWINGS_OTHER_CURRENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN CNT_DRAWINGS_POS_CURRENT ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Approved_CNT_DRAWINGS_POS_CURRENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Approved' THEN CNT_INSTALMENT_MATURE_CUM ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Approved_CNT_INSTALMENT_MATURE_CUM,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN MONTHS_BALANCE ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Refused_MONTHS_BALANCE,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN AMT_BALANCE ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Refused_AMT_BALANCE,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN AMT_CREDIT_LIMIT_ACTUAL ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Refused_AMT_CREDIT_LIMIT_ACTUAL,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN AMT_DRAWINGS_ATM_CURRENT ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Refused_AMT_DRAWINGS_ATM_CURRENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN AMT_DRAWINGS_CURRENT ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Refused_AMT_DRAWINGS_CURRENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN AMT_DRAWINGS_OTHER_CURRENT ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Refused_AMT_DRAWINGS_OTHER_CURRENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN AMT_DRAWINGS_POS_CURRENT ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Refused_AMT_DRAWINGS_POS_CURRENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN AMT_INST_MIN_REGULARITY ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Refused_AMT_INST_MIN_REGULARITY,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN AMT_PAYMENT_CURRENT ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Refused_AMT_PAYMENT_CURRENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN AMT_PAYMENT_TOTAL_CURRENT ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Refused_AMT_PAYMENT_TOTAL_CURRENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN AMT_RECEIVABLE_PRINCIPAL ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Refused_AMT_RECEIVABLE_PRINCIPAL,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN AMT_RECIVABLE ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Refused_AMT_RECIVABLE,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN AMT_TOTAL_RECEIVABLE ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Refused_AMT_TOTAL_RECEIVABLE,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN CNT_DRAWINGS_ATM_CURRENT ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Refused_CNT_DRAWINGS_ATM_CURRENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN CNT_DRAWINGS_CURRENT ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Refused_CNT_DRAWINGS_CURRENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN CNT_DRAWINGS_OTHER_CURRENT ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Refused_CNT_DRAWINGS_OTHER_CURRENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN CNT_DRAWINGS_POS_CURRENT ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Refused_CNT_DRAWINGS_POS_CURRENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Refused' THEN CNT_INSTALMENT_MATURE_CUM ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Refused_CNT_INSTALMENT_MATURE_CUM,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Sent proposal' THEN MONTHS_BALANCE ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Sent_proposal_MONTHS_BALANCE,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Sent proposal' THEN AMT_BALANCE ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Sent_proposal_AMT_BALANCE,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Sent proposal' THEN AMT_CREDIT_LIMIT_ACTUAL ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Sent_proposal_AMT_CREDIT_LIMIT_ACTUAL,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Sent proposal' THEN AMT_DRAWINGS_ATM_CURRENT ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Sent_proposal_AMT_DRAWINGS_ATM_CURRENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Sent proposal' THEN AMT_DRAWINGS_CURRENT ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Sent_proposal_AMT_DRAWINGS_CURRENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Sent proposal' THEN AMT_DRAWINGS_OTHER_CURRENT ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Sent_proposal_AMT_DRAWINGS_OTHER_CURRENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Sent proposal' THEN AMT_DRAWINGS_POS_CURRENT ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Sent_proposal_AMT_DRAWINGS_POS_CURRENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Sent proposal' THEN AMT_INST_MIN_REGULARITY ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Sent_proposal_AMT_INST_MIN_REGULARITY,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Sent proposal' THEN AMT_PAYMENT_CURRENT ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Sent_proposal_AMT_PAYMENT_CURRENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Sent proposal' THEN AMT_PAYMENT_TOTAL_CURRENT ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Sent_proposal_AMT_PAYMENT_TOTAL_CURRENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Sent proposal' THEN AMT_RECEIVABLE_PRINCIPAL ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Sent_proposal_AMT_RECEIVABLE_PRINCIPAL,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Sent proposal' THEN AMT_RECIVABLE ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Sent_proposal_AMT_RECIVABLE,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Sent proposal' THEN AMT_TOTAL_RECEIVABLE ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Sent_proposal_AMT_TOTAL_RECEIVABLE,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Sent proposal' THEN CNT_DRAWINGS_ATM_CURRENT ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Sent_proposal_CNT_DRAWINGS_ATM_CURRENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Sent proposal' THEN CNT_DRAWINGS_CURRENT ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Sent_proposal_CNT_DRAWINGS_CURRENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Sent proposal' THEN CNT_DRAWINGS_OTHER_CURRENT ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Sent_proposal_CNT_DRAWINGS_OTHER_CURRENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Sent proposal' THEN CNT_DRAWINGS_POS_CURRENT ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Sent_proposal_CNT_DRAWINGS_POS_CURRENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Sent proposal' THEN CNT_INSTALMENT_MATURE_CUM ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Sent_proposal_CNT_INSTALMENT_MATURE_CUM,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Demand' THEN MONTHS_BALANCE ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Demand_MONTHS_BALANCE,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Demand' THEN AMT_BALANCE ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Demand_AMT_BALANCE,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Demand' THEN AMT_CREDIT_LIMIT_ACTUAL ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Demand_AMT_CREDIT_LIMIT_ACTUAL,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Demand' THEN AMT_DRAWINGS_ATM_CURRENT ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Demand_AMT_DRAWINGS_ATM_CURRENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Demand' THEN AMT_DRAWINGS_CURRENT ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Demand_AMT_DRAWINGS_CURRENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Demand' THEN AMT_DRAWINGS_OTHER_CURRENT ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Demand_AMT_DRAWINGS_OTHER_CURRENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Demand' THEN AMT_DRAWINGS_POS_CURRENT ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Demand_AMT_DRAWINGS_POS_CURRENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Demand' THEN AMT_INST_MIN_REGULARITY ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Demand_AMT_INST_MIN_REGULARITY,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Demand' THEN AMT_PAYMENT_CURRENT ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Demand_AMT_PAYMENT_CURRENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Demand' THEN AMT_PAYMENT_TOTAL_CURRENT ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Demand_AMT_PAYMENT_TOTAL_CURRENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Demand' THEN AMT_RECEIVABLE_PRINCIPAL ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Demand_AMT_RECEIVABLE_PRINCIPAL,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Demand' THEN AMT_RECIVABLE ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Demand_AMT_RECIVABLE,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Demand' THEN AMT_TOTAL_RECEIVABLE ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Demand_AMT_TOTAL_RECEIVABLE,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Demand' THEN CNT_DRAWINGS_ATM_CURRENT ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Demand_CNT_DRAWINGS_ATM_CURRENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Demand' THEN CNT_DRAWINGS_CURRENT ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Demand_CNT_DRAWINGS_CURRENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Demand' THEN CNT_DRAWINGS_OTHER_CURRENT ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Demand_CNT_DRAWINGS_OTHER_CURRENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Demand' THEN CNT_DRAWINGS_POS_CURRENT ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Demand_CNT_DRAWINGS_POS_CURRENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Demand' THEN CNT_INSTALMENT_MATURE_CUM ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Demand_CNT_INSTALMENT_MATURE_CUM,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Signed' THEN MONTHS_BALANCE ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Signed_MONTHS_BALANCE,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Signed' THEN AMT_BALANCE ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Signed_AMT_BALANCE,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Signed' THEN AMT_CREDIT_LIMIT_ACTUAL ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Signed_AMT_CREDIT_LIMIT_ACTUAL,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Signed' THEN AMT_DRAWINGS_ATM_CURRENT ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Signed_AMT_DRAWINGS_ATM_CURRENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Signed' THEN AMT_DRAWINGS_CURRENT ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Signed_AMT_DRAWINGS_CURRENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Signed' THEN AMT_DRAWINGS_OTHER_CURRENT ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Signed_AMT_DRAWINGS_OTHER_CURRENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Signed' THEN AMT_DRAWINGS_POS_CURRENT ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Signed_AMT_DRAWINGS_POS_CURRENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Signed' THEN AMT_INST_MIN_REGULARITY ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Signed_AMT_INST_MIN_REGULARITY,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Signed' THEN AMT_PAYMENT_CURRENT ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Signed_AMT_PAYMENT_CURRENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Signed' THEN AMT_PAYMENT_TOTAL_CURRENT ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Signed_AMT_PAYMENT_TOTAL_CURRENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Signed' THEN AMT_RECEIVABLE_PRINCIPAL ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Signed_AMT_RECEIVABLE_PRINCIPAL,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Signed' THEN AMT_RECIVABLE ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Signed_AMT_RECIVABLE,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Signed' THEN AMT_TOTAL_RECEIVABLE ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Signed_AMT_TOTAL_RECEIVABLE,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Signed' THEN CNT_DRAWINGS_ATM_CURRENT ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Signed_CNT_DRAWINGS_ATM_CURRENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Signed' THEN CNT_DRAWINGS_CURRENT ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Signed_CNT_DRAWINGS_CURRENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Signed' THEN CNT_DRAWINGS_OTHER_CURRENT ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Signed_CNT_DRAWINGS_OTHER_CURRENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Signed' THEN CNT_DRAWINGS_POS_CURRENT ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Signed_CNT_DRAWINGS_POS_CURRENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Signed' THEN CNT_INSTALMENT_MATURE_CUM ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Signed_CNT_INSTALMENT_MATURE_CUM,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Completed' THEN MONTHS_BALANCE ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Completed_MONTHS_BALANCE,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Completed' THEN AMT_BALANCE ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Completed_AMT_BALANCE,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Completed' THEN AMT_CREDIT_LIMIT_ACTUAL ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Completed_AMT_CREDIT_LIMIT_ACTUAL,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Completed' THEN AMT_DRAWINGS_ATM_CURRENT ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Completed_AMT_DRAWINGS_ATM_CURRENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Completed' THEN AMT_DRAWINGS_CURRENT ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Completed_AMT_DRAWINGS_CURRENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Completed' THEN AMT_DRAWINGS_OTHER_CURRENT ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Completed_AMT_DRAWINGS_OTHER_CURRENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Completed' THEN AMT_DRAWINGS_POS_CURRENT ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Completed_AMT_DRAWINGS_POS_CURRENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Completed' THEN AMT_INST_MIN_REGULARITY ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Completed_AMT_INST_MIN_REGULARITY,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Completed' THEN AMT_PAYMENT_CURRENT ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Completed_AMT_PAYMENT_CURRENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Completed' THEN AMT_PAYMENT_TOTAL_CURRENT ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Completed_AMT_PAYMENT_TOTAL_CURRENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Completed' THEN AMT_RECEIVABLE_PRINCIPAL ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Completed_AMT_RECEIVABLE_PRINCIPAL,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Completed' THEN AMT_RECIVABLE ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Completed_AMT_RECIVABLE,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Completed' THEN AMT_TOTAL_RECEIVABLE ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Completed_AMT_TOTAL_RECEIVABLE,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Completed' THEN CNT_DRAWINGS_ATM_CURRENT ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Completed_CNT_DRAWINGS_ATM_CURRENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Completed' THEN CNT_DRAWINGS_CURRENT ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Completed_CNT_DRAWINGS_CURRENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Completed' THEN CNT_DRAWINGS_OTHER_CURRENT ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Completed_CNT_DRAWINGS_OTHER_CURRENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Completed' THEN CNT_DRAWINGS_POS_CURRENT ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Completed_CNT_DRAWINGS_POS_CURRENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Completed' THEN CNT_INSTALMENT_MATURE_CUM ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Completed_CNT_INSTALMENT_MATURE_CUM,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Active' THEN MONTHS_BALANCE ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Active_MONTHS_BALANCE,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Active' THEN AMT_BALANCE ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Active_AMT_BALANCE,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Active' THEN AMT_CREDIT_LIMIT_ACTUAL ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Active_AMT_CREDIT_LIMIT_ACTUAL,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Active' THEN AMT_DRAWINGS_ATM_CURRENT ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Active_AMT_DRAWINGS_ATM_CURRENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Active' THEN AMT_DRAWINGS_CURRENT ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Active_AMT_DRAWINGS_CURRENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Active' THEN AMT_DRAWINGS_OTHER_CURRENT ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Active_AMT_DRAWINGS_OTHER_CURRENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Active' THEN AMT_DRAWINGS_POS_CURRENT ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Active_AMT_DRAWINGS_POS_CURRENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Active' THEN AMT_INST_MIN_REGULARITY ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Active_AMT_INST_MIN_REGULARITY,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Active' THEN AMT_PAYMENT_CURRENT ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Active_AMT_PAYMENT_CURRENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Active' THEN AMT_PAYMENT_TOTAL_CURRENT ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Active_AMT_PAYMENT_TOTAL_CURRENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Active' THEN AMT_RECEIVABLE_PRINCIPAL ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Active_AMT_RECEIVABLE_PRINCIPAL,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Active' THEN AMT_RECIVABLE ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Active_AMT_RECIVABLE,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Active' THEN AMT_TOTAL_RECEIVABLE ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Active_AMT_TOTAL_RECEIVABLE,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Active' THEN CNT_DRAWINGS_ATM_CURRENT ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Active_CNT_DRAWINGS_ATM_CURRENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Active' THEN CNT_DRAWINGS_CURRENT ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Active_CNT_DRAWINGS_CURRENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Active' THEN CNT_DRAWINGS_OTHER_CURRENT ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Active_CNT_DRAWINGS_OTHER_CURRENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Active' THEN CNT_DRAWINGS_POS_CURRENT ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Active_CNT_DRAWINGS_POS_CURRENT,
AVG(CASE WHEN NAME_CONTRACT_STATUS= 'Active' THEN CNT_INSTALMENT_MATURE_CUM ELSE 0 END) AS AVG_NAME_CONTRACT_STATUS_Active_CNT_INSTALMENT_MATURE_CUM
FROM
  credit_card_balance
GROUP BY
  SK_ID_CURR
""")


In [106]:
df_credit_card_balance_and_name_contract_status_num.createOrReplaceTempView("df_credit_card_balance_and_name_contract_status_num")

# **Analisando quantas novas variáveis explicativas nós conseguimos criar após as etapas anteriores**

In [107]:
# Conta o número de colunas na tabela
num_colunas_base = len(credit_card_balance.columns)
num_colunas_base_cat = len(df_credit_card_balance_cat.columns)
num_colunas_base_num = len(df_credit_card_balance_num.columns)
num_colunas_base_cat_e_num = len(df_credit_card_balance_and_name_contract_status_num.columns)

# Imprime o resultado
print("A base original credit_card_balance", num_colunas_base, "colunas.")
print('')
print("A base categórica da credit_card_balance possui", num_colunas_base_cat, " novas colunas.")
print('')
print("A base numérica da credit_card_balance possui", num_colunas_base_num, " novas colunas.")
print('')
print("A base numérica com categórica da credit_card_balance possui", num_colunas_base_cat_e_num, " novas colunas.")
print('')

A base original credit_card_balance 23 colunas.

A base categórica da credit_card_balance possui 29  novas colunas.

A base numérica da credit_card_balance possui 73  novas colunas.

A base numérica com categórica da credit_card_balance possui 505  novas colunas.



In [109]:
df_application_train = spark.read.csv("/content/drive/MyDrive/hackathon-pod-ds/application_train.csv",sep=',',header=True, encoding='utf-8', inferSchema=True)
df_application_train.createOrReplaceTempView("df_application_train")

# **Juntando os dataframes criados com a application train**

In [116]:
df_junto = df_application_train.join(df_credit_card_balance_cat, "SK_ID_CURR",  how = 'left')\
               .join(df_credit_card_balance_num, "SK_ID_CURR",  how = 'left')\
               .join(df_credit_card_balance_and_name_contract_status_num, "SK_ID_CURR",  how = 'left')

In [112]:
df_junto.show(5)

+----------+------+------------------+-----------+------------+---------------+------------+----------------+----------+-----------+---------------+---------------+--------------------+--------------------+--------------------+-------------------+--------------------------+----------+-------------+-----------------+---------------+-----------+----------+--------------+---------------+----------------+----------+----------+---------------+---------------+--------------------+---------------------------+--------------------------+-----------------------+--------------------------+--------------------------+---------------------------+----------------------+----------------------+-----------------------+-----------------+------------------+------------------+------------------+--------------+----------------+---------------------------+---------------+--------------+-------------+-------------+-------------+-------------+------------+--------------------+--------------+-------------------

# **Validando se não houve inclusão de novas linhas no dataset original (application train)**

In [117]:
df_junto.count()

215257

In [114]:
df_application_train.count()

215257